# Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install requirements

In [2]:
!pip install pytorch_lightning
!pip install timm

     |████████████████████████████████| 527 kB 5.1 MB/s 
     |████████████████████████████████| 133 kB 71.5 MB/s 
     |████████████████████████████████| 829 kB 52.6 MB/s 
     |████████████████████████████████| 596 kB 30.5 MB/s 
     |████████████████████████████████| 397 kB 64.4 MB/s 
     |████████████████████████████████| 952 kB 53.7 MB/s 
     |████████████████████████████████| 1.1 MB 52.2 MB/s 
     |████████████████████████████████| 94 kB 2.3 MB/s 
     |████████████████████████████████| 271 kB 73.2 MB/s 
     |████████████████████████████████| 144 kB 68.1 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=0683cd73cfd57e34234986bb9a49d04ce01889daf1f7c077a0fbb46bd6a1e071
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successful

     |████████████████████████████████| 431 kB 5.1 MB/s 


# Import relevant libraries

In [3]:
import sys
import os
module_path = os.path.join(os.path.abspath(''), "drive/MyDrive/Master_Thesis_Profactor/zdmp/")
if module_path not in sys.path:
  sys.path.append(module_path)

import zdmp
from utils.lightning_classifier import Classifier

import timm
import torch
import torchvision
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

# Train Luna Model

In [4]:

def save_model(model, name):
    path = os.path.join(
        "/content/drive/MyDrive/Master_Thesis_Profactor/zdmp/pretrained_weights",
        f"{name}.pth.tar"
    )
    torch.save(model.state_dict(), path)

In [14]:
model_names = ["luna", "alexnet", "resnet", "convnext", "efficientnetv2"]
model_name = model_names[4]

In [ ]:
model = timm.models.swin_tiny_patch4_window7_224(pretrained=True)

Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth" to /root/.cache/torch/hub/checkpoints/swin_tiny_patch4_window7_224.pth


In [6]:
model_2 = timm.models.convnext.convnext_tiny(pretrained=True)

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny_1k_224_ema.pth


In [8]:
model_2.head.fc

Linear(in_features=768, out_features=1000, bias=True)

In [ ]:
model_3 = timm.models.efficientnetv2_rw_s(pretrained=True)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_v2s_ra2_288-a6477665.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2s_ra2_288-a6477665.pth


In [ ]:
print(model)
print(model_2)
print(model_3)

SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): Sequential(
    (0): BasicLayer(
      dim=96, input_resolution=(56, 56), depth=2
      (blocks): ModuleList(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path): Identity()
          (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (a

In [16]:
# Make the results reproducible
pl.seed_everything(42)

# Prepare dataset
ds_train = zdmp.get_train_data()
ds_test = zdmp.get_test_data()

# Prepare Dataloaders
dl_train = DataLoader(ds_train, batch_size=32, num_workers=2, shuffle=True)
dl_test = DataLoader(ds_test, batch_size=32, num_workers=2)

# Prepare model
if model_name == "luna":
    model = zdmp.get_luna(pretrained=False)
if model_name == "alexnet":
    model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
    model.classifier[6] = torch.nn.Linear(4096, 2)
elif model_name == "resnet":
    model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
    model.fc = torch.nn.Linear(512, 2)
elif model_name == "convnext":
    model = timm.models.convnext.convnext_tiny(pretrained=True)
    model.head.fc = torch.nn.Linear(768, 2)
elif model_name == "efficientnetv2":
    model = timm.models.efficientnetv2_rw_s(pretrained=True)
    model.classifier = torch.nn.Linear(1792, 2)

# Prepare Classifier to train the model
classifier = Classifier(model)

# Criteria to stop early
#early_stopping_callback = EarlyStopping(monitor="val_loss", min_delta=0.00, patience=5)

# Prepare the Trainer
trainer = pl.Trainer(gpus=1, max_epochs=3)
#trainer = pl.Trainer(gpus=1, min_epochs=25, max_epochs=100)

# Find the best learning rate
#trainer.tune(classifier, train_dataloaders=dl_train, val_dataloaders=dl_test)

# Train
trainer.fit(classifier, dl_train, dl_test)

# Save the trained weights
save_model(classifier.model, model_name)

Global seed set to 42


zdmp - ViT size - False
get_data - ViT size - False
transform - ViT size - False


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_v2s_ra2_288-a6477665.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2s_ra2_288-a6477665.pth
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | model         | EfficientNet     | 22.2 M
1 | train_metrics | MetricCollection | 0     
2 | valid_metrics | MetricCollection | 0     
---------------------------------------------------
22.2 M    Trainable params
0         Non-trainable params
22.2 M    Total params
88.608    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# Tensorboard
For both train and validation:
* Accuracy
* Precission
* Recall
* Loss

In [17]:
!tensorboard dev upload --logdir '/content/lightning_logs'

Upload started and will continue reading any new data as it's added to the logdir.

To stop uploading, press Ctrl-C.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/ubKwkJGhQBKCrNv7da2Jnw/

[2022-02-12T13:24:18] Started scanning logdir.
[2022-02-12T13:24:19] Total uploaded: 154 scalars, 12 tensors (72 B), 0 binary objects
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/main.py", line 46, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 312, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 258, in _run_main
    sys.exit(main(argv))
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/program.py", line 276, in main
    return runner(self.flags) or 0
  File "/usr/local/lib/pyth

# Test trained model

In [20]:
wrong = 0
w = "*"*10
r = ""
for i, (img, lbl) in enumerate(ds_test):
    out_ = model(img.unsqueeze(0)).softmax(1)
    out = int(out_.softmax(1).argmax())
    if out != lbl:
        wrong+=1
    print(f"{i}\tPred: {out}\tTrue:{lbl}\t{out_}\t{r if out==lbl else w}")
print(f"Wrong: {wrong}/{len(ds_test)}")

0	Pred: 0	True:1	tensor([[0.7256, 0.2744]], grad_fn=<SoftmaxBackward0>)	**********
1	Pred: 0	True:1	tensor([[0.7377, 0.2623]], grad_fn=<SoftmaxBackward0>)	**********
2	Pred: 0	True:1	tensor([[0.7610, 0.2390]], grad_fn=<SoftmaxBackward0>)	**********
3	Pred: 0	True:1	tensor([[0.7513, 0.2487]], grad_fn=<SoftmaxBackward0>)	**********
4	Pred: 0	True:0	tensor([[0.7755, 0.2245]], grad_fn=<SoftmaxBackward0>)	
5	Pred: 0	True:0	tensor([[0.7791, 0.2209]], grad_fn=<SoftmaxBackward0>)	
6	Pred: 0	True:0	tensor([[0.7787, 0.2213]], grad_fn=<SoftmaxBackward0>)	
7	Pred: 0	True:0	tensor([[0.7931, 0.2069]], grad_fn=<SoftmaxBackward0>)	
8	Pred: 0	True:1	tensor([[0.7341, 0.2659]], grad_fn=<SoftmaxBackward0>)	**********
9	Pred: 0	True:0	tensor([[0.7814, 0.2186]], grad_fn=<SoftmaxBackward0>)	
10	Pred: 0	True:0	tensor([[0.7773, 0.2227]], grad_fn=<SoftmaxBackward0>)	
11	Pred: 0	True:1	tensor([[0.7544, 0.2456]], grad_fn=<SoftmaxBackward0>)	**********


KeyboardInterrupt: ignored

In [21]:
trainer.validate(dataloaders=dl_test)

Restoring states from the checkpoint path at /content/lightning_logs/version_3/checkpoints/epoch=2-step=623.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /content/lightning_logs/version_3/checkpoints/epoch=2-step=623.ckpt


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'val_Accuracy': 0.9972028136253357,
 'val_Precision': 0.9972028136253357,
 'val_Recall': 0.9972028136253357,
 'val_loss': 0.01029963232576847}
--------------------------------------------------------------------------------


[{'val_Accuracy': 0.9972028136253357,
  'val_Precision': 0.9972028136253357,
  'val_Recall': 0.9972028136253357,
  'val_loss': 0.01029963232576847}]